train.csv: A full training dataset with the following attributes:

id: unique id for a news article
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks the article as potentially unreliable
1: unreliable
0: reliable

test.csv: A testing training dataset with all the same attributes at train.csv without the label.

Importing the Dependencies


In [2]:
import numpy as np
import pandas as pd
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
print(stopwords.words('english'))
# printing the stopwords in English

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Data Pre-Processing

In [3]:
# loading the dataset to a pandas Dataframe
news_train_dataset = pd.read_csv('/content/drive/MyDrive/ML Data Files/fake news train.csv')
news_test_dataset = pd.read_csv('/content/drive/MyDrive/ML Data Files/fake news test.csv')

news_train_dataset.shape
news_test_dataset.shape

news_train_dataset.head
news_test_dataset.head

# counting the number of missing values in the dataset
news_train_dataset.isnull().sum()
news_test_dataset.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [4]:
# replacing the null values with empty string
news_train_dataset = news_train_dataset.fillna('')
news_test_dataset = news_test_dataset.fillna('')


In [ ]:
# merging the author name and news title
news_train_dataset['content'] = news_train_dataset['author']+' '+news_train_dataset['title']
news_test_dataset['content'] = news_test_dataset['author']+' '+news_test_dataset['title']

print(news_train_dataset['content'])
print(news_test_dataset['content'])

In [ ]:
# seperating the data & label
# X_train = news_train_dataset.drop(columns='label', axis = 1)
# Y_train = news_train_dataset['label']

# print(X_train)
# print(Y_train)

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [6]:
port_stem = PorterStemmer()

In [7]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  # remove all numbers and punctuations & only keep the alphabets a-z, A-Z
  # if there are numbers and punctuations then replace with ' '
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [8]:
news_train_dataset['content'] = news_train_dataset['content'].apply(stemming)
news_test_dataset['content'] = news_test_dataset['content'].apply(stemming)

In [9]:
print(news_train_dataset['content'])
print(news_test_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object
0       david streitfeld specter trump loosen tongu pu...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victor tim tebow attempt anoth comeback...
4        truth broadcast network keiser report meme war e
                 

In [10]:
# seperating the data and label
X = news_train_dataset['content'].values
Y = news_train_dataset['label'].values

X_unknown_test = news_test_dataset['content'].values

print(X)
print(Y)
print(X.shape)

print(X_unknown_test)
X_unknown_test.shape

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]
(20800,)
['david streitfeld specter trump loosen tongu purs string silicon valley new york time'
 'russian warship readi strike terrorist near aleppo'
 'common dream nodapl nativ american leader vow stay winter file lawsuit polic'
 ... 'mike mcphate california today exactli sushi new york time'
 'us marin deploy russian border norway'
 'teddi wayn awkward sex onscreen new york time']


(5200,)

Converting texual data to numerical data

In [11]:
vectorizer = TfidfVectorizer()
# Tf-->Term Frequency, idf-->inverse document frequency
# Tf counts the number of times a word is repeating in a document and Vectorizer assigns a numerical value to it according to importance
# idf finds those words that are repeating so many times that it does not add value/significance and assigns a according numerical value
vectorizer.fit(X)
vectorizer.fit(X_unknown_test)

X = vectorizer.transform(X)
X_unknown_test = vectorizer.transform(X_unknown_test)
# vectorizer.transform will convert all these assigned numerical values and turn them into FEATURE VECTORS

In [12]:
print(X)
#print(X_unknown_test)

  (0, 8442)	0.30637649390862737
  (0, 7212)	0.2909577136736758
  (0, 4632)	0.34214314503179805
  (0, 4169)	0.3121403148732912
  (0, 3805)	0.25509319068205105
  (0, 2691)	0.2772114975963559
  (0, 2051)	0.3210623876107006
  (0, 1570)	0.2715587684340332
  (0, 1303)	0.4337415027138284
  (0, 146)	0.3163982964753294
  (1, 8995)	0.31382147886721357
  (1, 3700)	0.1945215363335672
  (1, 2999)	0.7133615317876577
  (1, 1931)	0.2651920557432443
  (1, 1483)	0.19469666733924623
  (1, 1155)	0.36142956889835864
  (1, 978)	0.15437033055741373
  (1, 784)	0.303475905694002
  (2, 8401)	0.42489598472948564
  (2, 5161)	0.4951808375391314
  (2, 3264)	0.3594826818588411
  (2, 2935)	0.38263854579603196
  (2, 1668)	0.4478645702386819
  (2, 1558)	0.31310798733727463
  (3, 8583)	0.25621216601337693
  :	:
  (20797, 7037)	0.2479405960124089
  (20797, 6604)	0.28761420145284444
  (20797, 6494)	0.24609677725734458
  (20797, 5528)	0.07987188194883543
  (20797, 5148)	0.173601869788456
  (20797, 5113)	0.30876252310723
  

Splitting the Training data and Test data

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(16640, 9142)
(16640,)
(4160, 9142)
(4160,)


Training the Model: Logistic Regression

In [14]:
model = LogisticRegression()

In [15]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [16]:
# accuracy score on the training data
X_train_pred = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_pred, Y_train)

print('Accuracy Score of the training data: ', training_data_accuracy)

Accuracy Score of the training data:  0.9856370192307692


In [18]:
# accuracy score on the test data
X_test_pred = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_pred, Y_test)

print('Accuracy Score of the test data: ', test_data_accuracy)

Accuracy Score of the test data:  0.9798076923076923


Making a Prediction System

In [19]:
#X_unknown_test

prediction = model.predict(X_unknown_test)
data = {'Test': news_test_dataset['id'],
        'Pred': prediction}
df = pd.DataFrame(data)
print(df)

from google.colab import files

df.to_csv('MY_results.csv', encoding = 'utf-8-sig')
files.download('MY_results.csv')


       Test  Pred
0     20800     0
1     20801     1
2     20802     1
3     20803     0
4     20804     1
...     ...   ...
5195  25995     0
5196  25996     0
5197  25997     0
5198  25998     1
5199  25999     0

[5200 rows x 2 columns]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>